<a href="https://colab.research.google.com/github/ajeevansiva/stock_news_summarizer/blob/main/News_Analysis_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install accelerate
!pip install bitsandbytes
!pip install pyngrok
!pip install flask_cors

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 48.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [2]:
from transformers import AutoTokenizer, AutoModel, pipeline, AutoModelForSeq2SeqLM
from transformers import LlamaForCausalLM
# Specify the name of the pre-trained model
model_name = 'daryl149/llama-2-7b-chat-hf'


# Load the pre-trained model and its tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = LlamaForCausalLM.from_pretrained(pretrained_model_name_or_path='daryl149/llama-2-7b-chat-hf', load_in_8bit=True, device_map={'': 0},)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

...

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [3]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
import requests
from flask_cors import CORS
import cv2
from transformers import pipeline
from bs4 import BeautifulSoup
import time

In [8]:
port_no = 5000
summarization_pipeline = pipeline("summarization", model='Ajeevan123/NewsSummarization', tokenizer='Ajeevan123/NewsSummarization')

app = Flask(__name__)
CORS(app)
ngrok.set_auth_token("2eggMwe5lgs8Zd2rLOJhMMhtvFw_4XEfW5xVm6n6gUkgqsboz")
public_url =  ngrok.connect(port_no).public_url

@app.route("/")
def home():
    return f"Running Flask on Google Colab!"

print(f"To acces the Gloable link please click {public_url}")

To acces the Gloable link please click https://bf15-34-125-188-217.ngrok-free.app


In [9]:
@app.route('/getUrl', methods=['POST'])
def generate_response():
    global sr, summary_sentiment_info
    data = request.json
    url = data.get('link', '')

    print(data)
    # Define headers
    headers = {
      "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36"
    }

    # Send a GET request to the URL with headers
    r = requests.get(url, headers=headers)
    r.raise_for_status()  # Raise an error for bad status codes

    # Parse the HTML content of the page
    soup = BeautifulSoup(r.text, 'html.parser')

    # Find all paragraphs and filter out messages
    paragraphs = soup.find_all('p')
    filtered_paragraphs = [p.text.strip() for p in paragraphs if "Thank you for your patience." not in p.text.strip()]

    newsString = ' '.join(filtered_paragraphs)

    summarized_output = summarization_pipeline(newsString, max_length=510, min_length=40, do_sample=False)

    sr = str(summarized_output[0]['summary_text'])

    sentiment = pipeline('sentiment-analysis')

    result = sentiment(sr)

    summary_sentiment_info = "Based on the description and the sentiment analysis answer the below prompts: Summary: " + sr + ", Sentiment: " + result[0]['label'] + ", Score: " + str(result[0]['score'])

    print(summary_sentiment_info)

    return jsonify({"response": result})


@app.route('/chatbotResponse', methods=['POST'])
def chatbot_interaction():
    data = request.json
    prompt = data.get('prompt', '')


    # Combine user prompt and summary sentiment info
    instruction = f"### Instruction:\n{summary_sentiment_info}\n{prompt}\n### Response:\n"


    print(instruction)

    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=512)

    # Generate the response
    result = pipe(instruction)

    response = result[0]['generated_text'][len(instruction):]

    return jsonify({
        "chatbot_response": response
    })

if __name__ == '__main__':
    app.run(port=port_no)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


{'link': 'https://www.dailymirror.lk/top-story/24-suspects-died-in-police-custody-last-year/155-280221'}


Token indices sequence length is longer than the specified maximum sequence length for this model (818 > 512). Running this sequence through the model will result in indexing errors
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
INFO:werkzeug:127.0.0.1 - - [05/Apr/2024 16:07:06] "POST /getUrl HTTP/1.1" 200 -


Based on the description and the sentiment analysis answer the below prompts: Summary: Sri Lanka Human Rights Commission (SLHRC) has reported that 24 suspects had died in police custody in the last two months . Reply Prison Friday, 05 April 2024 07:55 PM THIS ACCOUNT OFFICIALLY. This is needed. If I may suggest Police wearing body cameras will ease this situation. The traffic cops also must wear body cameras that will record images and the conversations ., Sentiment: NEGATIVE, Score: 0.872887134552002
### Instruction:
Based on the description and the sentiment analysis answer the below prompts: Summary: Sri Lanka Human Rights Commission (SLHRC) has reported that 24 suspects had died in police custody in the last two months . Reply Prison Friday, 05 April 2024 07:55 PM THIS ACCOUNT OFFICIALLY. This is needed. If I may suggest Police wearing body cameras will ease this situation. The traffic cops also must wear body cameras that will record images and the conversations ., Sentiment: NEGA

INFO:werkzeug:127.0.0.1 - - [05/Apr/2024 16:07:23] "POST /chatbotResponse HTTP/1.1" 200 -


In [3]:
import requests
from bs4 import BeautifulSoup
import time

def scrape_news(url):
    try:
        # Define headers
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36"
        }

        # Send a GET request to the URL with headers
        r = requests.get(url, headers=headers)
        r.raise_for_status()  # Raise an error for bad status codes

        # Parse the HTML content of the page
        soup = BeautifulSoup(r.text, 'html.parser')

        # Find all paragraphs and filter out messages
        paragraphs = soup.find_all('p')
        filtered_paragraphs = [p.text.strip() for p in paragraphs if "Thank you for your patience." not in p.text.strip()]

        return filtered_paragraphs
    except Exception as e:
        print("Error occurred:", e)
        return None


url = "https://www.dailymirror.lk/breaking-news/Tourist-rescued-after-falling-down-Sri-Pada-precipice/108-279832" #Edit url here

news = scrape_news(url)
newsString = ' '.join(news)
if news:
    print("News headlines:")
    for headline in news:
        print("-", headline)
else:
    print("No news scraped.")


News headlines:
- Last Updated : 2024-04-05 20:55:00
- Daily FT
- Sunday Times
- Mirror Edu
- Tamil Mirror
- Lankadeepa
- Middleast Lankadeepa
- Ada
- Deshaya
- Life Online
- Hi Online
- HitAd
- TimesJobs
- E-Paper
- Home delivery
- Advertise with us
- Mobile Apps
- feedback
- Archive
- Print Ads
- Fri, 05 Apr 2024 Today's Paper
- 29 March 2024 03:14 pm
  - 11      - {{hitsCtrl.values.hits}}
- By Ranjith Rajapakshe
- Ratnapura, March 29 (Daily Mirror)- An Indian tourist who fell down a more than 100 metre precipice at the Ehela Kanuwa, Ratnapura-Sri Pada route in the early hours of this morning was rescued by a team from the Maskeliya Police Special Task Force (STF) camp.
- Police said the tourist Bharath Chandradas (25) from Mumbai Maharastra had been trekking to Sri Pada along the Ratnapura route with several others when he fell off the protective fence down the precipice at about 5.00 am today.
- On information from the Udamaluwa police post, the STF rushed to the scene and rescued 

In [4]:
from transformers import pipeline

def summarize_text(input_text, model_name="Ajeevan123/NewsSummarization", max_chunk_len=510):
    summarization_pipeline = pipeline("summarization", model=model_name, tokenizer=model_name)


    summarized_output = summarization_pipeline(input_text, max_length=510, min_length=40, do_sample=False)

    return summarized_output

# Example input text
input_text = newsString

# Summarize the input text
summary = summarize_text(input_text)
print(summary)
sr = str(summary[0]['summary_text'])
print(type(summary))
# Print the summary
print("Summary:")
print(sr)

config.json:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1352 > 512). Running this sequence through the model will result in indexing errors


[{'summary_text': 'Bharath Chandradas (25) from Mumbai Maharastra had been trekking to Sri Pada along the Ratnapura route with several others when he fell off the protective fence down the precipice at about 5.00 am today . on information from the Udamaluwa police post, the STF rushed to the scene and rescued the tourist and provided first aid . he was carried to Nallathanniya and from there admitted to hospital .'}]
<class 'list'>
Summary:
Bharath Chandradas (25) from Mumbai Maharastra had been trekking to Sri Pada along the Ratnapura route with several others when he fell off the protective fence down the precipice at about 5.00 am today . on information from the Udamaluwa police post, the STF rushed to the scene and rescued the tourist and provided first aid . he was carried to Nallathanniya and from there admitted to hospital .


In [5]:
from transformers import pipeline
sentiment = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [6]:
result = sentiment(sr)
summary_sentiment_info = "Based on the description and the sentiment analysis answer the below prompts: Summary: " + sr + ", Sentiment: " + result[0]['label'] + ", Score: " + str(result[0]['score'])

In [7]:
# User's prompt
user_prompt = "From which country is this person"


# Combine user prompt and summary sentiment info
instruction = f"### Instruction:\n{summary_sentiment_info}\n{user_prompt}\n### Response:\n"

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=512)

# Generate the response
result = pipe(instruction)

response = result[0]['generated_text'][len(instruction):]

print(response)

Based on the information provided in the prompt, the person, Bharath Chandradas, is from India, specifically from Mumbai, Maharashtra. Therefore, the answer to the question is: India.
